### This jupyter notebook employs a fully connective neural network(FC) or its alias artificial neural network (ANN) to learn the mapping between demanded magnetic field between current configuration

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import torch
from early_stopping import EarlyStopping

if torch.cuda.device_count():
    device = 'cuda'
    use_gpu = True
    print('Good to go')
else:
    device = 'cpu'
    use_gpu = False
    print('Using cpu')

In [ ]:
from ReadData import ReadCurrentAndField 
import glob
import os 

# print(os.getcwd())
foldername="./Data/"
filepattern = "MagneticField[0-9]*.txt"
#data = ReadFolder(foldername,filepattern)
load_file_num = 1460
train_file_num = 1000
grid_size = 21
data = ReadCurrentAndField (foldername,filepattern, load_file_num)


data=data.reshape(load_file_num,grid_size,grid_size,grid_size,18)
mask = torch.cat((torch.ones(1,1,1,1,12),1e3*torch.ones(1,1,1,1,6)), dim=4)
# position unit mm, B field unit mT, Current unit Ampere
data = mask*data

sparsity = 4

Position_Bfield =data[:train_file_num,0::sparsity,0::sparsity,0::sparsity,12:].reshape(-1,6) 
Current = data[:train_file_num,0::sparsity,0::sparsity,0::sparsity,:12].reshape(-1,12) 

print(np.abs(Position_Bfield[:,3:]).mean())
print(data.shape)
print('position Bfield shape', Position_Bfield.shape)
print('Current shape', Current.shape)


In [ ]:
from Neural_network import NN_net, Plain_fc_block, weight_init, eMNS_Dataset
from Training_loop_v2 import train_ANN
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer, TorchConfig
from ray.tune.tuner import Tuner
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import ray, os
import torch.nn.functional as F

# construct dataset
dataset = eMNS_Dataset(
    x=Position_Bfield,
    y=Current
)

# split the dataset to train, validation, test
train_set, valid_set = torch.utils.data.random_split(dataset, [0.9,0.1])

# normailzation
extremes = dataset.train_norm_ANN(train_indices = train_set.indices, boundary_index=3)

###############################################
# Config the neural network
###############################################
num_input = 6
num_output = 12
fc_stages = [(num_input,100,1),(100,50,1),(50,25,1)]
fc_network = NN_net(None,fc_stages,None,Plain_fc_block, num_output=num_output)


model_path = r"./Trained_model/EMS_ANN_v2.pt"
forward_model = torch.load(model_path)['model'].to(device)
forward_model.eval()

loss_func = lambda current, Bfield, position: F.mse_loss(current, torch.zeros_like(current)) + F.l1_loss(Bfield, forward_model(torch.cat((current, position), axis =1)))


################################################
# Train the neural network
################################################

train_loop_config = {
                'epochs': 50,
                'lr_max': 1e-3,
                'lr_min': 2.5e-5,
                'batch_size': 128,
                'L2_norm'   : 0,
                'verbose': False,
                'schedule': [],
                'learning_rate_decay': 0.5,
                'num_input'   : num_input,
                'num_output'  : num_output,
                'fc_stages'   : fc_stages,
                'backward'    : True,
                'maxB'        : extremes[2],
                'minB'        : extremes[3],
                'device'      : device,
                'train_set'   : train_set,
                'valid_set'   : valid_set,
                'loss_func'   : loss_func,
                'forward_model': forward_model
                # You can even grid search various datasets in Tune.
                # "datasets": tune.grid_search(
                #         [ds1, ds2]
                #     ),
}

scaling_config = ScalingConfig(
    num_workers = 1,
    use_gpu = use_gpu,
    resources_per_worker={"CPU":1, "GPU":1}
)

run_config = RunConfig(name="EMS_ANN_backward_v2",checkpoint_config=CheckpointConfig(num_to_keep=1))


# for windows
# torch_config = TorchConfig(backend="nccl")
trainer = TorchTrainer(
    train_loop_per_worker = train_ANN,
    train_loop_config = train_loop_config,
    scaling_config = scaling_config,
    run_config = run_config,
    # torch_config= torch_config # for windows

)
result = trainer.fit()



In [ ]:
from torchsummary import summary
summary(fc_network, (1,6))
for param_tensor in fc_network.state_dict():
    print(param_tensor, '\t', fc_network.state_dict()[param_tensor].size())

In [ ]:
from utils import plot_ray_results
plot_ray_results(results=result, metrics_names = ['rmse_val','rmse_train'])

In [ ]:
checkpoint_data = torch.load(os.path.join(result.checkpoint.path,"model.pt"))

model_path = r"./Trained_model/EMS_ANN_backward_v2.pt"
torch.save(checkpoint_data, model_path)

## Test dataset performance

In [ ]:
# position unit mm, current unit Ampere, B field unit mT
sparsity = 4
# Position_Bfield_test =data[train_file_num:, ::sparsity, ::sparsity, ::sparsity, :6].reshape(-1,6) 

# Current_test = data[train_file_num:, ::sparsity, ::sparsity, ::sparsity,:12].reshape(-1,12)

Position_Bfield_test =data[:train_file_num, ::sparsity, ::sparsity, ::sparsity, :6].reshape(-1,6) 

Current_test = data[:train_file_num, ::sparsity, ::sparsity, ::sparsity,:12].reshape(-1,12)

num_sample = Position_Bfield_test.shape[0]
print('position Bfield shape', Position_Bfield_test.shape)
print('Current shape', Current_test.shape)

# construct dataset
test_set = eMNS_Dataset(
    x=Position_Bfield_test,
    y=Current_test
)
test_set.test_norm_ANN(extremes=extremes, boundary_index=3)

test_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=train_loop_config['batch_size'],
    shuffle=False)


In [ ]:
from utils import predict_check_rmse_ANN, check_rmse_ANN
from Training_loop_v2 import construct_model_ANN 

model_path = r"./Trained_model/EMS_ANN_backward_v2.pt"
backward_model = torch.load(model_path)['model']

prediction, rmse, mse, Rsquare = predict_check_rmse_ANN(test_loader, backward_model, config=train_loop_config)
# check_rmse_ANN(test_loader, backward_model, config=train_loop_config)

In [ ]:
from utils import denorm
from Neural_network import NN_net, Plain_fc_block, weight_init, eMNS_Dataset
from Training_loop_v2 import train_ANN
from ray.train import RunConfig, ScalingConfig, CheckpointConfig
from ray.train.torch import TorchTrainer
from ray.tune.tuner import Tuner
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import ray, os
import torch.nn.functional as F

# construct dataset
dataset = eMNS_Dataset(
    x=Position_Bfield,
    y=Current
)

# split the dataset to train, validation, test
train_set, valid_set = torch.utils.data.random_split(dataset, [0.9,0.1])

# normailzation
extremes = dataset.train_norm_ANN(train_indices = train_set.indices, boundary_index=3)
print(extremes[2],extremes[3])
denorm(dataset.x[:,3:],extremes[2],extremes[3],device='cpu').abs().mean()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
grid_test = int(np.ceil(grid_size/sparsity))
B_est = prediction.reshape(-1, grid_test, grid_test, grid_test, 3)
Bfield_test = Position_Bfield[:,3:].reshape(-1, grid_test, grid_test, grid_test, 3)

current_index=6
z_plane_index= 2
# fig, ax = plt.subplots(3, 2)
# fig.tight_layout(h_pad=2)
fig = plt.figure()
ylables=['Bx\mT','By\mT','Bz\mT']
# fig.tight_layout(pad=0.4, w_pad=0, h_pad=0)
for i in range(1,4):
    plt.subplot(3,2,2*i-1)
    plt.imshow(B_est[current_index,:,:,z_plane_index,i-1])    
    plt.colorbar()
    plt.ylabel(ylables[i-1])
    plt.subplot(3,2,2*i)
    plt.imshow(Bfield_test[current_index,:,:,z_plane_index,i-1])
    plt.colorbar()
plt.show()

In [ ]:
import numpy as np 
A = np.arange(21)
print(A[::4])
print(A[::1])

In [ ]:
A = torch.arange(3) + 0.0
B = A.reshape(1,3)
print(F.mse_loss(B, torch.zeros_like(B)))
print(torch.linalg.norm(B))

In [ ]:
np.sqrt(5)